# Titanic Dataset

In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
import seaborn as sns
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score


In [100]:
titanic=pd.read_csv('/kaggle/input/campus-valley/Titanic.txt', 
                    sep=',', quotechar='"', encoding='ISO 8859-15')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,ParCh,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Basic EDA 

In [101]:
titanic =titanic.drop([918,920]) 

In [102]:
titanic.Survived.value_counts() 

Survived
0.0    584
1.0    361
Name: count, dtype: int64

In [103]:
titanic['Pclass']=titanic['Pclass'].astype(str).astype(int)

In [104]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 949
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  948 non-null    int64  
 1   Survived     945 non-null    float64
 2   Pclass       948 non-null    int64  
 3   Name         948 non-null    object 
 4   Sex          948 non-null    object 
 5   Age          763 non-null    float64
 6   SibSp        948 non-null    int64  
 7   ParCh        948 non-null    int64  
 8   Ticket       948 non-null    object 
 9   Fare         948 non-null    float64
 10  Cabin        217 non-null    object 
 11  Embarked     946 non-null    object 
dtypes: float64(3), int64(4), object(5)
memory usage: 96.3+ KB


In [133]:
cols = ['Name','Ticket','Cabin','PassengerId']
titanic_DT= titanic
titanic_DT = titanic_DT.drop(cols,axis=1)
titanic_DT.head(2)

,Survived,Pclass,Sex,Age,SibSp,ParCh,Fare,Embarked
0,0.0,3,male,22.0,1,0,7.2500,S
1,1.0,1,female,38.0,1,0,71.2833,C


### Now we convert the Pclass, Sex, Embarked to columns in pandas and drop them after conversion.

In [106]:
dummies = []
cols = ['Pclass','Sex','Embarked']
for col in cols:
 dummies.append(pd.get_dummies(titanic_DT[col]))

In [107]:
titanic_dummies = pd.concat(dummies, axis=1)
titanic_dummies.head()

,1,2,3,female,male,C,Q,S
0,False,False,True,False,True,False,False,True
1,True,False,False,True,False,True,False,False
2,False,False,True,True,False,False,False,True
3,True,False,False,True,False,False,False,True
4,False,False,True,False,True,False,False,True


### Then  we concatenate to the original dataframe columnwise

In [108]:
titanic_DT = pd.concat((titanic_DT,titanic_dummies),axis=1)

In [109]:
titanic_DT.head()

,Survived,Pclass,Sex,Age,SibSp,ParCh,Fare,Embarked,1,2,3,female,male,C,Q,S
0,0.0,3,male,22.0,1,0,7.2500,S,False,False,True,False,True,False,False,True
1,1.0,1,female,38.0,1,0,71.2833,C,True,False,False,True,False,True,False,False
2,1.0,3,female,26.0,0,0,7.9250,S,False,False,True,True,False,False,False,True
3,1.0,1,female,35.0,1,0,53.1000,S,True,False,False,True,False,False,False,True
4,0.0,3,male,35.0,0,0,8.0500,S,False,False,True,False,True,False,False,True


### Now that we converted Pclass, Sex, Embarked values into columns, we drop the redundant same columns from the dataframe

In [110]:
titanic_DT = titanic_DT.drop(['Pclass','Sex','Embarked'],axis=1)
titanic_DT.head()

,Survived,Age,SibSp,ParCh,Fare,1,2,3,female,male,C,Q,S
0,0.0,22.0,1,0,7.2500,False,False,True,False,True,False,False,True
1,1.0,38.0,1,0,71.2833,True,False,False,True,False,True,False,False
2,1.0,26.0,0,0,7.9250,False,False,True,True,False,False,False,True
3,1.0,35.0,1,0,53.1000,True,False,False,True,False,False,False,True
4,0.0,35.0,0,0,8.0500,False,False,True,False,True,False,False,True


In [111]:
titanic_DT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 949
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  945 non-null    float64
 1   Age       763 non-null    float64
 2   SibSp     948 non-null    int64  
 3   ParCh     948 non-null    int64  
 4   Fare      948 non-null    float64
 5   1         948 non-null    bool   
 6   2         948 non-null    bool   
 7   3         948 non-null    bool   
 8   female    948 non-null    bool   
 9   male      948 non-null    bool   
 10  C         948 non-null    bool   
 11  Q         948 non-null    bool   
 12  S         948 non-null    bool   
dtypes: bool(8), float64(3), int64(2)
memory usage: 51.8 KB


In [112]:
titanic_DT['Survived'] = titanic_DT['Survived'].interpolate()

In [113]:
titanic_DT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 949
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  948 non-null    float64
 1   Age       763 non-null    float64
 2   SibSp     948 non-null    int64  
 3   ParCh     948 non-null    int64  
 4   Fare      948 non-null    float64
 5   1         948 non-null    bool   
 6   2         948 non-null    bool   
 7   3         948 non-null    bool   
 8   female    948 non-null    bool   
 9   male      948 non-null    bool   
 10  C         948 non-null    bool   
 11  Q         948 non-null    bool   
 12  S         948 non-null    bool   
dtypes: bool(8), float64(3), int64(2)
memory usage: 51.8 KB


# Model Training

In [114]:
y = titanic_DT['Survived'].values #output

In [115]:
titanic_DT.head()

,Survived,Age,SibSp,ParCh,Fare,1,2,3,female,male,C,Q,S
0,0.0,22.0,1,0,7.2500,False,False,True,False,True,False,False,True
1,1.0,38.0,1,0,71.2833,True,False,False,True,False,True,False,False
2,1.0,26.0,0,0,7.9250,False,False,True,True,False,False,False,True
3,1.0,35.0,1,0,53.1000,True,False,False,True,False,False,False,True
4,0.0,35.0,0,0,8.0500,False,False,True,False,True,False,False,True


In [116]:
titanic_ml= titanic_DT.drop(['Survived',"Age"],axis=1)
X = titanic_ml.values
titanic_ml.head()

,SibSp,ParCh,Fare,1,2,3,female,male,C,Q,S
0,1,0,7.2500,False,False,True,False,True,False,False,True
1,1,0,71.2833,True,False,False,True,False,True,False,False
2,0,0,7.9250,False,False,True,True,False,False,False,True
3,1,0,53.1000,True,False,False,True,False,False,False,True
4,0,0,8.0500,False,False,True,False,True,False,False,True


In [117]:
xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=0.30,random_state=0)

# Linear Regression

In [118]:
from sklearn.linear_model import  LinearRegression

In [119]:
modelLinearRegression = LinearRegression()
modelLinearRegression.fit(xtrain,ytrain)


LinearRegression()

In [120]:
ypredLinearRegression=modelLinearRegression.predict(xtest)

In [121]:
print(f"The Mean Absolute Error is {mean_absolute_error(y_test,ypredLinearRegression)}")
print(f"The Mean Squared Error   is {mean_squared_error(y_test,ypredLinearRegression)}")

The Mean Absolute Error is 0.3096578545449614
The Mean Squared Error   is 0.15141126766035531


## User Input

In [122]:
result=modelLinearRegression.predict([[1,0,7.2500,False,False,True,False,True,False,False,True]])
if result:
    print(f"The Pasenger Survied")
else:
    print("Pasagnger does not survived")

The Pasenger Survied


# Random Forest

In [123]:
from sklearn.ensemble import RandomForestRegressor

In [124]:
ModelRandomForest = RandomForestRegressor()
ModelRandomForest.fit(xtrain,ytrain)


RandomForestRegressor()

In [125]:
ypredRadomForest=ModelRandomForest.predict(xtest)

In [126]:
print(f"The Mean Absolute Error is {mean_absolute_error(y_test,ypredRadomForest)}")
print(f"The Mean Squared Error   is {mean_squared_error(y_test,ypredRadomForest)}")


The Mean Absolute Error is 0.2547174575840363
The Mean Squared Error   is 0.14711065331196396


## User Input

In [127]:
result=ModelRandomForest.predict([[1,0,7.2500,False,False,True,False,True,False,False,True]])
if result:
    print(f"The Pasenger Survied")
else:
    print("Pasagnger does not survived")

Pasagnger does not survived


# Decision Tree

In [128]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score


In [129]:
ModelDecisionTree = tree.DecisionTreeClassifier(max_depth=4)
ModelDecisionTree.fit(xtrain,ytrain)


DecisionTreeClassifier(max_depth=4)

In [130]:
ypredDecionTree=clf.predict(xtest)

In [131]:
print(f"The Mean Absolute Error is {mean_absolute_error(y_test,ypredDecionTree)}")
print(f"The Mean Squared Error   is {mean_squared_error(y_test,ypredDecionTree)}")
print(f"The Accuracy Score  is {accuracy_score(y_test,ypredDecionTree)*100}")

The Mean Absolute Error is 0.19649122807017544
The Mean Squared Error   is 0.19649122807017544
The Accuracy Score  is 80.35087719298247


# User Input

In [132]:
result=ModelDecisionTree.predict([[1,0,7.2500,False,False,True,False,True,False,False,True]])
if result:
    print(f"The Pasenger Survied")
else:
    print("Pasagnger does not survived")

Pasagnger does not survived
